<a href="https://colab.research.google.com/github/cecorreas/AI-For-Beginners/blob/main/Tarea_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 2

### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Felipe Villena](https://fabianvillena.cl/).
- Profesor Auxiliar: [Gabriel Iturra](https://giturra.cl/)

### Instrucciones generales

- Grupos de máximo 3 personas.
- Esta prohibido compartir las respuestas con otros grupos.
- Indicios de copia serán penalizados con la nota mínima.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente, si utiliza material extra debe citarlo.


### Integrantes

> POR FAVOR AGREGAR TODOS LOS NOMBRES DE LOS INTEGRANTES

## Contexto

El discurso de odio es cualquier expresión que promueva o incite a la discriminación, la hostilidad o la violencia hacia una persona o grupo de personas en una relación asimétrica de poder, tal como la raza, la etnia, el género, la orientación sexual, la religión, la nacionalidad, una discapacidad u otra característica similar.

En cambio, la incivilidad se refiere a cualquier comportamiento o actitud que rompe las normas de respeto, cortesía y consideración en la interacción entre personas. Esta puede manifestarse de diversas formas, tal como insultos, ataques personales, sarcasmo, desprecio, entre otras.

En esta tarea tendrán a su disposición un dataset de textos con las etiquetas `odio`, `incivilidad` o `normal`. La mayor parte de los datos se encuentra en español de Chile. Con estos datos, deberán entrenar un modelo que sea capaz de predecir la etiqueta de un texto dado.

El corpus para esta tarea se compone de 3 datasets:  
- [Multilingual Resources for Offensive Language Detection de Arango et al. (2022)](https://aclanthology.org/2022.woah-1.pdf#page=136)
- [Dataton UTFSM No To Hate (2022)](http://dataton.inf.utfsm.cl/)
- Datos generados usando la [API de GPT3 (modelo DaVinci 03)](https://platform.openai.com/docs/models/gpt-3).

Agradecimientos a los autores por compartir los datos y a David Miranda, Fabián Diaz, Santiago Maass y Jorge Ortiz por revisar y reetiquetar los datos en el contexto del curso "Taller de Desarrollo de Proyectos de IA" (CC6409), Departamento de Ciencias de la Computación, Universidad de Chile.

Los datos solo pueden ser usados con fines de investigación y docencia. Está prohibida la difusión externa.


## Tarea a resolver

Para esta tarea, buscaremos desarrollar un *benchmark* sobre una tarea de clasificación de NLP. Un benchmark es básicamente utilizar diferentes técnicas para resolver una misma tarea específica, en este caso seguiremos buscando alternativas para resolver un problema de clasificación. Particularmente, se le pide:

- Utilizar una técnica de word embeddings.
- Implementar una arquitectura en RNN utilizando PyTorch.
- Utilizar transformers para revolver el problema de clasificación, en especifico utilizar BETO.
- Utilizar algún LLM utilizando Zero y Few short learning para resolver el problema de clasificación.


### Cargar el dataset


En esta sección, cargaremos el dataset desde el repositorio del módulo. Para ello ejecute las siguientes líneas:

In [1]:
import pandas as pd

In [2]:
# Dataset.
dataset_df = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")


### Analizar los datos

En esta sección analizaremos el balance de los datos. Para ello se imprime la cantidad de tweets de cada dataset agrupados por la intensidad de sentimiento.

In [3]:
dataset_df.sample(5)

,id,texto,clase
8751,7035,"Esto es asqueroso.\nA dedo, lo ponen en un car...",incivilidad
346,11885,@user @user @user @user @user @user Chilw está...,odio
3130,10548,Lucha de consejos: CDE busca frenar orden del ...,incivilidad
3510,1445,"#PautaLibre\nLa hermana de Luksic, dueña de 30...",normal
7792,10564,@user Solo los venezolanos de carácter humilde...,odio


In [4]:
dataset_df["clase"].value_counts()

clase
incivilidad    5424
normal         4280
odio           2510
Name: count, dtype: int64

In [5]:
target_classes = list(dataset_df['clase'].unique())
target_classes

['normal', 'incivilidad', 'odio']

### Instalar librerias

Puede usar esta celda para instalar las librerías que estime necesario.

In [42]:
%%capture
from sklearn.model_selection import train_test_split
import spacy.lang.es
import tensorflow as tf
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin

### Importar librerías

En esta sección, importamos la liberías necesarias para el correcto desarrollo de esta tarea. Puede utilizar otras librerías que no se en encuentran aquí, pero debe citar su fuente.

In [16]:
import nltk
import numpy as np

from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin

# importe aquí sus clasificadores

import matplotlib.pyplot as plt

# word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser

# Pytorch imports
import torch
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

from torch import nn
from torch.nn import functional as F

from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

from torch.optim import Adam


### Separar el dataset

Su primer ejercicio es partir el dataset en train y test usando `scikit-learn`. El como procese el dataset para el input de los modelos queda a su criterio.

In [54]:
#  'clase' es la variable objetivo
X = dataset_df.drop(['clase','id'], axis=1)
y = dataset_df['clase']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de prueba:", len(X_test))
print("Tamaño del conjunto de entrenamiento:", len(y_train))
print("Tamaño del conjunto de prueba:", len(y_test))

Tamaño del conjunto de entrenamiento: 9771
Tamaño del conjunto de prueba: 2443
Tamaño del conjunto de entrenamiento: 9771
Tamaño del conjunto de prueba: 2443


### Sparse Vectors

#### Crear representaciones de texto

Como hemos mencionado en las clases anteriores, los modelos de Machine Learning no son capaces de entender el texto directamente para resolver cualquier tarea. Es vital realizar un paso intermedio, que es buscar un mecanismo que permita traducir el texto a representaciones que puedan ser entendidas por los modelos de ML, y que conserven las propiedades semánticas y sintacticas del lenguaje. Es por esto, que en está sección estudiaremos los métodos de text representation estudiados.

- a) El primer método que se les solicita, es crear una representación de Bag of Words utilizando la librería `scikit-learn`. Para esto, puede serle útil revisar el tutorial tres.

In [9]:
stopwords = spacy.lang.es.stop_words.STOP_WORDS # La biblioteca Spacy tiene una lista de stopwords en español


In [22]:
count_vectorizer.fit(X_train["texto"])

CountVectorizer(max_df=0.05, min_df=2,
                stop_words=['durante', 'mio', 'haceis', 'nada', 'usa', 'vez',
                            'usas', 'gran', 'contra', 'da', 'algunas', 'mucho',
                            'me', 'eran', 'cuál', 'haya', 'salvo', 'dieron',
                            'dice', 'ésas', 'donde', 'debe', 'también', 'o',
                            'aquella', 'el', 'poca', 'nadie', 'mediante',
                            'está', ...])

In [24]:
text_vectorized = count_vectorizer.transform(X_train["texto"])

In [25]:
text_vectorized.shape

(9771, 10258)

In [28]:
X_train["texto"][8995]

'L@user gays son abominables, no tienen derecho a existir en esta sociedad. ¡Tienen que desaparecer! ¡Son una abominación, no queremos verlos en nuestras calles!'

In [32]:

documento = [X_train["texto"][8995]]
vector = count_vectorizer.transform(documento).toarray()
vector

array([[0, 0, 0, ..., 0, 0, 0]])

- b) El segundo método que se les solicita, es crear una representación de TF-IDF utilizando la librería `scikit-learn`. Para esto, puede serle útil revisar el tutorial tres.

In [35]:
def get_word_scores(text,vectorizer):
    """A partir de un texto y un vectorizador retorna los puntajes asignados a cada palabra del texto"""
    feature_names = list({k: v for k, v in sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1])}.keys()) # Vocabulario
    doc = vectorizer.transform([text]) # Vectorizamos el texto de entrada
    idxs = np.argwhere(doc)[:,1] # Extraemos los índices donde sí hay palabras representadas
    words = [feature_names[i] for i in idxs] # Extraemos las palabras asociadas a los índices extraídos
    scores = np.array(doc.todense())[0][idxs] # Extraemos los puntajes asociadas a los índices extraídos
    return list(reversed(sorted(zip(words,scores),key=lambda tup: tup[1]))) # Retornamos una lista de palabras y puntajes

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Ahora puedes crear una instancia de TfidfVectorizer directamente
tfidf_vectorizer = TfidfVectorizer(
    stop_words = list(stopwords),
    max_df = 0.05,
    min_df = 2
)

tfidf_vectorizer.fit(X_train["texto"])

TfidfVectorizer(max_df=0.05, min_df=2,
                stop_words=['durante', 'mio', 'haceis', 'nada', 'usa', 'vez',
                            'usas', 'gran', 'contra', 'da', 'algunas', 'mucho',
                            'me', 'eran', 'cuál', 'haya', 'salvo', 'dieron',
                            'dice', 'ésas', 'donde', 'debe', 'también', 'o',
                            'aquella', 'el', 'poca', 'nadie', 'mediante',
                            'está', ...])

In [37]:
get_word_scores(X_train["texto"][8995],count_vectorizer)

[('verlos', 1),
 ('sociedad', 1),
 ('gays', 1),
 ('existir', 1),
 ('desaparecer', 1),
 ('derecho', 1),
 ('calles', 1),
 ('abominación', 1),
 ('abominables', 1)]

In [39]:
get_word_scores(X_train["texto"][8995],tfidf_vectorizer)

[('abominables', 0.4258365344978351),
 ('verlos', 0.38613758877274273),
 ('desaparecer', 0.36088366824611323),
 ('abominación', 0.34188617271030625),
 ('existir', 0.3393529293456477),
 ('gays', 0.3056594514553463),
 ('calles', 0.3033162837632017),
 ('derecho', 0.2592557176144077),
 ('sociedad', 0.2337422197533822)]

#### Definir clasificadores

En esta parte, se procedera a entrenar, los clasificadores por cada representación de texto, creada en la parte anterior, para esto usted debe:

Se sugiere revisar el tutorial 4, para recordar como utilizar los clasificadores de `scikit-learn`.

1. Definir los datos y las etiquetas de entrenamiento y testeo:

In [ ]:
#esto ya fue definido mas arriba en el codigo
#X_train = ...
#y_train = ...

#X_test = ...
#y_test = ...

2. Definir al menos dos clasificadores por representación:

Una pregunta, que podría hacerse: ¿Es necesario separar el dataset en training y testing, usando alguna función de `scikit-learn` en este caso? Fundamente:
R: esto ya fue realizado mas arriba en el codigo



In [56]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
# limpiar puntuaciones y separar por tokens.
punctuation = string.punctuation + "«»“”‘’…—"

In [40]:
class Doc2VecTransformer(BaseEstimator, TransformerMixin):
    """ Transforma tweets a representaciones vectoriales usando algún modelo de Word Embeddings.
    """

    def __init__(self, model, aggregation_func):
        # extraemos los embeddings desde el objeto contenedor. ojo con esta parte.
        self.model = model.wv

        # indicamos la función de agregación (np.min, np.max, np.mean, np.sum, ...)
        self.aggregation_func = aggregation_func

    def simple_tokenizer(self, doc, lower=False):
        """Tokenizador. Elimina signos de puntuación, lleva las letras a minúscula(opcional) y
           separa el tweet por espacios.
        """
        if lower:
            doc.translate(str.maketrans('', '', string.punctuation)).lower().split()
        return doc.translate(str.maketrans('', '', string.punctuation)).split()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        doc_embeddings = []

        for doc in X:
            # tokenizamos el documento. Se llevan todos los tokens a minúscula.
            # ojo con esto, ya que puede que tokens con minúscula y mayúscula tengan
            # distintas representaciones
            tokens = self.simple_tokenizer(doc, lower = True)

            selected_wv = []
            for token in tokens:
                if token in self.model.index_to_key:
                    selected_wv.append(self.model[token])

            # si seleccionamos por lo menos un embedding para el tweet, lo agregamos y luego lo añadimos.
            if len(selected_wv) > 0:
                doc_embedding = self.aggregation_func(np.array(selected_wv), axis=0)
                doc_embeddings.append(doc_embedding)
            # si no, añadimos un vector de ceros que represente a ese documento.
            else:
                print('No pude encontrar ningún embedding en el tweet: {}. Agregando vector de ceros.'.format(doc))
                doc_embeddings.append(np.zeros(self.model.vector_size)) # la dimension del modelo

        return np.array(doc_embeddings)


#### Bag of Bow

In [45]:
import multiprocessing
w2v = Word2Vec(min_count=10,
                      window=4,
                      vector_size=200,
                      sample=6e-5,
                      alpha=0.03,
                      min_alpha=0.0007,
                      negative=20,
                      workers=multiprocessing.cpu_count())

In [46]:
clf1 = LogisticRegression(max_iter=1000000)

doc2vec_mean = Doc2VecTransformer(w2v, np.mean)
doc2vec_sum = Doc2VecTransformer(w2v, np.sum)
doc2vec_max = Doc2VecTransformer(w2v, np.max)


pipeline = Pipeline([('doc2vec', doc2vec_sum), ('clf1', clf1)])

In [57]:
assert len(X_train) == len(y_train), "Las longitudes de 'X_train' y 'y_train' no coinciden"
pipeline.fit(X_train, y_train)

No pude encontrar ningún embedding en el tweet: texto. Agregando vector de ceros.


ValueError: Found input variables with inconsistent numbers of samples: [1, 9771]

No pude encontrar ningún embedding en el tweet: texto. Agregando vector de ceros.


ValueError: Found input variables with inconsistent numbers of samples: [1, 9771]

In [50]:
import string
y_pred = pipeline.predict(X_test)


No pude encontrar ningún embedding en el tweet: texto. Agregando vector de ceros.


NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
clf2 = ...

#### TF-IDF

In [ ]:
clf3 = ...

In [ ]:
clf4 = ...

Para esta sección pueden utilizar cualquier clasificadores que estimen conveniente.

3. Entrenar cada uno de los clasificadores desarrollados.

In [ ]:
# Entrene cada uno de los sus clasificadores

### Evaluar los clasificadores.

En esta sección, se espera que entregue la matriz de confusión y el reporte de clasificación de los clasificadores en la sección pasada. Por lo que seria útil estudiar las funciones `confusion_matrix` y `classification_report` de `scikit-lear`. Podrá encontrar un ejemplo en el tutorial 4.

In [ ]:
# Evalue cada uno de sus clasificadores

Finalmente, ¿Qué pude decir del rendimiento de todos los clasificadores?, ¿Cree que alguna representación pudo resolver mejor la tarea? Jusfique, se espera que de un análsis para cada uno de los 4 clasificadores, identificado sus aciertos y fallas.

### Word Embeddings

Tal como vimos en el tutorial 4, en esta pregunta se busca que cree representaciones basadas en Word Embeddings. Particularmente, debe seguir los siguientes pasos:

#### Crear sus representaciones

En esta parte debe crear representación del conjunto entrenamiento usando algún método de Word Embeddings (cualquiera que estime conveniente). Se recomienda revisar el tutorial 4, y la documentación de la librería `gensim`. Recuerde considerar cada uno de los pasos que vimos en clases previas, llamar un tokenizador, crear el vocabulario, llamar a un detector de frases, etc.

In [ ]:
# Su código

#### Crear representaciones a nivel de oración

Recuerde que los Word Embeddings actuan a nivel de palabras, por lo que es necesario definir una clase reconstruya la oración utilizados los embeddings del paso anterior utilizando alguna función de agregación. Puede revisar el tutorial 4. Se le recomienda llenar la siguiente clase.

In [ ]:
class Doc2VecTransformer(BaseEstimator, TransformerMixin):
    """ Transforma tweets a representaciones vectoriales usando algún modelo de Word Embeddings.
    """

    def __init__(self, model, aggregation_func, ...): # Puede agregar más parametros si lo estima necesario
        # Completar
        pass

    def simple_tokenizer(self, doc, lower=False):
        # puede definir su propio tokenizador
        pass
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # completar
        pass

#### Clasificación utilizando Word Emebdding.

Cree al menos dos experimentos de clasificación, ya sea variando el clasificador o los hiperparametros asociados.

#### Reporte de evaluación

Genere un reporte de los resultados para ambos experimentos, utilizando una matriz de confusión y el report de clasificación de `scikit-learn`. Considere realizarlo en el dataset de testing.

In [ ]:
# reporte 1

In [ ]:
# reporte 2

Analice los resultados, ¿porqué cree que estuvo esos resultados?, ¿Hubo una variación considerable entre ambos experimentos?. Justique.

### Crear un clasificador basado en RNN

En esta parte de le pide definir un clasificador utilizando `PyTorch` con alguna arquitectura en Redes Recurrentes. Para ello debe realizar todos los pasos vistos en el tutorial 5, por lo que se recomienda revisarlo. Importante, no puede replicar ningún ejemplo de los del tutorial 5, debe proponer sus propias arquitecturas. Se le recomienda leer como utilizar variaciones de la RNN, como la LSTM o GRU en `Pytorch`.

Para completa esta parte deberá replicar el flujo de trabajo de como utilizar `PyTorch`. Esta esctrictamente prohibido utilizar variaciones que resuelvan directamente este problema, como `PyTorch Lightning` o `TensorFlow`. Los pasos a completar son los siguientes:

#### Cargar el dataset.

#### Definir el vocabulario.

#### Cargar el `DataLoader`

Recuerde que podría necesitar una función intermedia para procesar cada batch durante el entrenamiento, pero no es obligatorio hacerlo.

#### Definir la red recurrente.

Recuerde que debe difirnir los hyperparametros que estime conveniente.

In [ ]:
class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()
        pass
    def forward(self, X_batch):
        pass

#### Funciones de entrenamiento y evaluación.

Para esta parte, puede utilizar las funciones vista en el tutorial directamente. Pero es su reponsabilidad ajustarlas a su código.

In [ ]:
def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))


def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
        CalcValLossAndAccuracy(model, loss_fn, val_loader)

def MakePredictions(model, loader):
    Y_shuffled, Y_preds = [], []
    for X, Y in loader:
        preds = model(X)
        Y_preds.append(preds)
        Y_shuffled.append(Y)
    gc.collect()
    Y_preds, Y_shuffled = torch.cat(Y_preds), torch.cat(Y_shuffled)

    return Y_shuffled.detach().numpy(), F.softmax(Y_preds, dim=-1).argmax(dim=-1).detach().numpy()



##### Entrenamiento del modelo

Ejecute el entrenamiento de su modelo propuesto.

In [ ]:
epochs = ...
learning_rate = ...

loss_fn = nn.CrossEntropyLoss()
rnn_classifier = RNNClassifier()
optimizer = Adam(rnn_classifier.parameters(), lr=learning_rate)

TrainModel(rnn_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

##### Evaluacion del modelo

Ejecute la evaluación de su modelo, y genere un reporte de evaluación similar al de la pregunta anterior.

In [ ]:
Y_actual, Y_preds = MakePredictions(rnn_classifier, test_loader)

In [ ]:
print("Test Accuracy : {}".format(accuracy_score(Y_actual, Y_preds)))
print("\nClassification Report : ")
print(classification_report(Y_actual, Y_preds, target_names=target_classes))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_actual, Y_preds))

Finalmente, análice sus resultados, ¿Porqué cree que obtuvo esos resultados?, ¿Es mejor que sólo utilizar Word Embeddings, porque?. Justique.

### Transformers BERT.

Para esta tarea se le piden crear una representación de texto usando la arquitectura basada en transformers, BERT:



In [ ]:
!pip install transformers


#### Import BETO

Para esto debe importar el tokenizador y el modelo BETO.

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

#### Ejemplo de extracción de features.

Luego, debe cargar los modelos pre-entrenados:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
model = AutoModelForMaskedLM.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', output_hidden_states=True)

Una vez cargado BETO, debe utilizarlo para extraer los embeddings asociados a la texto de su corpus. Se espera que usted realice lo siguiente:



Tokenizamos el texto para extraer los ids a cada palabra en el vocabulario interno de BETO, se recomienda utilizar el padding, trunctation, y max_length para considerar oraciones de diferentes tamaños.

Luego, debe verificar si cada uno de los ids extraídos se encuentran en la misma máquina donde fue cargado el modelo (CPU o GPU), se recomienda dejar todo en GPU.

In [ ]:
# oración
sentence = "Hola, que tal? me gusta mucho el curso de NLP"

# extraemos los ids de los tokens, se recomienda definir los valores de las variables:
#  padding, truncation, max_length debido a que las secuencia de texto puede tener diferentes largos
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", max_length=512)
# revisamos si cada de los ids, se encuentran en la misma máquina que el modelo (GPU o CPU)
inputs = {k: v.to(model.device) for k, v in inputs.items()}
inputs

{'input_ids': tensor([[    4,  1734,  1019,  1041,  1713,  1059,  1094,  2331,  1700,  1039,
           4430,  1009, 15567, 30968,     5]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Una vez, extraídos los ids, usted debe obtener los estados ocultos de las ultimas capas de BETO.

In [ ]:
# Extraemos la features
outputs = model(**inputs)

# ahora `outputs` tendrá el atributo `hidden_states`
hidden_states = outputs.hidden_states


Finalmente, debe guardar los embeddings en CPU los embeddings del token [CLS] que será usados en la clasificación del análisis de sentimientos.

In [ ]:
with torch.no_grad():
# Seleccionamos la última capa y obtenemos el primer token ([CLS]) para cada ejemplo
# Estos son los embeddings que normalmente se usan para tareas de clasificación.
  cls_embeddings = hidden_states[-1][:, 0, :].cpu().numpy()


#### Extraer features de todo el dataset

Considerando lo anterior, usted debe implementar la función `get_beto_features_in_batches` para extraer los features de BETO (los estados ocultos y los embeddings del token [CLS]).

Esta función recibe dos parámetros, el texto a vectorizar y un tamaño de batch, debido a que es extramadamente recomendable a que procesen el texto por lotes, ya que si cargan todos el modelo se les agotará la memoria RAM.

In [ ]:
# Función para procesar los textos en lotes y obtener las características de BETO

def get_beto_features_in_batches(texts, batch_size):

  pass

Ahora extraíga los features, un punto importante es que la extracción de features podría tomar alrededor de una a dos horas dependiendo del tamaño del batch que utilicen.

In [ ]:
train_embs = get_beto_features_in_batches(..., ...)

#### Clasificación

Una vez extraído los features de BETO, realice la clasificación de los embeddings obtenidos. Debe implementar dos clasificadores a su elección.

#### Reporte de evaluación

Una vez realizada la clasificación, realice el reporte de clasificación y el análsis de la matriz confusión para ambos clasificadores.

Recuerde que para hacer esto, debe extraer los features del dataset de testing.

Finalmente, que puede decir de los resultados obtenidos. ¿Es mejor que los dos métodos anteriores? ¿A que se debe esto? Justifue

### Large Language Model

##### Zero Shot Learning

Utilizando la técnica de zero shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM LlaMa.

##### Few Shot Learning

Utilizando la técnica de few shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM LlaMa.